In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf
from tensorflow.keras.models import load_model

2023-12-18 09:10:00.155590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import cv2, numpy, datetime, pygame, keyboard
import xlwings as xw
import serial
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [4]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [5]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [6]:
s= 2

i = 0

entered = False
al_entered = False




In [ ]:

arduino = serial.Serial('/dev/cu.usbmodem143201', 9600)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cap.set(cv2.CAP_PROP_FPS, 30)


#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day
start_time = None
running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False
    
    
    
    
    
    state, frame = cap.read()
    
    if state!=True:
        break
        
    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')
    
    if len(res[0]['identity'])>0:
        file_path = res[0]['identity'][0]
        parts = file_path.split('/')
        name = parts[-2] 
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        #liveness
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32, 32))  # Increase image resolution
        img = img.astype('float')/255.0

        # Apply data augmentation (optional)
        # img = augment_data(img)

        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)

        liveness = model.predict(img)
        liveness = liveness[0].argmax()
        
        
        

        
        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
            
            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)
            
            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)
            
            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)         
            
            
            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)
            # If face detected, start or update the timer
            if start_time is None:
                start_time = moment
                arduino.write(b'1')

            elif (moment - start_time).seconds > 2:
#                 arduino.write((name + '\n').encode())
                # If face is detected for more than 1 minute, update attendance
                if name not in students:
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                start_time = None

            
        else:
            arduino.write(b'0') 
                # Draw red rectangle for unknown faces
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1)
            cv2.rectangle(frame, (xmin, ymin - 25), (xmax, ymin), (255, 255, 255), -1)
            cv2.putText(frame, 'Unknown', (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
#cv2.destroyAllWindows()

2023-12-18 09:10:08.461 python[2491:100335] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


There are  9  representations found in  representations_vgg_face.pkl
find function lasts  1.9084808826446533  seconds
1/1 [==============================] - 0s 101ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.24374985694885254  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22843122482299805  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22382903099060059  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22010517120361328  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.23909282684326172  seconds
1/1 [=====================

find function lasts  0.22674202919006348  seconds
1/1 [==============================] - 0s 16ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.21514129638671875  seconds
1/1 [==============================] - 0s 16ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22945618629455566  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22426080703735352  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.21169495582580566  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.21309804916381836  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  rep

find function lasts  0.22079896926879883  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22900986671447754  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22571802139282227  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22503089904785156  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.23523497581481934  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2310018539428711  seconds
1/1 [==============================] - 0s 16ms/step
There are  9  representations found in  repr

1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2362198829650879  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22562408447265625  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2267751693725586  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22124791145324707  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22691702842712402  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.221

...

find function lasts  0.22393298149108887  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2219099998474121  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2250990867614746  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22321820259094238  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22673773765563965  seconds
1/1 [==============================] - 0s 19ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22960996627807617  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  repre

find function lasts  0.23125076293945312  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2319629192352295  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22233796119689941  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22296905517578125  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22281694412231445  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2199239730834961  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  repre

find function lasts  0.22535228729248047  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22476792335510254  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.23454785346984863  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22617602348327637  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2247309684753418  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.21804380416870117  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  repr

find function lasts  0.223923921585083  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22090601921081543  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22101497650146484  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.21894288063049316  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2309110164642334  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22826504707336426  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  repres

find function lasts  0.21641993522644043  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22452831268310547  seconds
1/1 [==============================] - 0s 19ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22173786163330078  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22283387184143066  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22209405899047852  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.223099946975708  seconds
1/1 [==============================] - 0s 17ms/step
There are  9  representations found in  repre